In [1]:
# ===== Import necessary libraries =====
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch.quantization
from torch.quantization import QConfig, default_observer, default_per_channel_weight_observer

import torch.nn.utils.prune as prune

import numpy as np



# ===== Set up the SimpleNN model =====

class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, quantTrue=False):
        super(CNNBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
    
    def prune_weights(self, current_sparsity, target_sparsity):
        remaining_weights = 1.0 - current_sparsity
        pruning_ratio = 1.0 - (target_sparsity / remaining_weights)
        if current_sparsity == 0.0:
            pruning_ratio = target_sparsity
        print("Pruning Ratio:" + str(pruning_ratio))
        prune.l1_unstructured(self.conv, 'weight', amount=pruning_ratio)


    def to_sparse(self):
        self.conv.weight = nn.Parameter(self.conv.weight.to_sparse())
    
class ResBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1, quantTrue=False):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion*out_channels, kernel_size=1, stride=stride, bias=False), nn.BatchNorm2d(self.expansion*out_channels)
            )
        
        self.quantadd = nn.quantized.FloatFunctional()

        self.quantTrue = quantTrue

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.quantTrue:
            if self.shortcut is not None:
                identity = self.shortcut(identity)
            out = self.quantadd.add(out, identity)
        else:
            if self.shortcut is not None:
                identity = self.shortcut(identity)
            out += identity

        out = F.relu(out)
        return out
    
    def prune_weights(self, current_sparsity, target_sparsity):
        remaining_weights = 1.0 - current_sparsity
        pruning_ratio = 1.0 - (target_sparsity / remaining_weights)
        if current_sparsity == 0.0:
            pruning_ratio = target_sparsity
        print("Pruning Ratio:" + str(pruning_ratio))
        prune.l1_unstructured(self.conv1, 'weight', amount=pruning_ratio)
        prune.l1_unstructured(self.conv2, 'weight', amount=pruning_ratio)

    def to_sparse(self):
        self.conv1.weight = nn.Parameter(self.conv1.weight.to_sparse())
        self.conv2.weight = nn.Parameter(self.conv2.weight.to_sparse())

class CNN(nn.Module):
    def __init__(self, architecture_type='cnn', quantTrue = False, conv_layer_configs=None, fc_layer_configs=None, res_block_configs=None, num_classes=10):
        super(CNN, self).__init__()
        
        # Initialization code for other parts of the class remains the same
        
        if architecture_type == 'cnn':
            self.myNetworkType = 'cnn'
            self.features = self._make_cnn_layers(conv_layer_configs)
            prev_features = conv_layer_configs[-1]['out_channels']
        elif architecture_type == 'resnet':
            self.myNetworkType = 'resnet'
            self.init_conv = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
            self.init_bn = nn.BatchNorm2d(16)
            self.init_relu = nn.ReLU(inplace=True)
            self.features, prev_features = self._make_resnet_layers(res_block_configs, quantTrue)
        else:
            raise ValueError("Unsupported architecture type: {}".format(architecture_type))
        
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = self._make_fc_layers(prev_features, fc_layer_configs, num_classes)

    def _make_cnn_layers(self, conv_layer_configs):
        layers = []
        for conv_layer in conv_layer_configs:
            in_channels = conv_layer['in_channels']
            out_channels = conv_layer['out_channels']
            kernel_size = conv_layer['kernel_size']
            stride = conv_layer['stride']
            padding = conv_layer['padding']
            block = CNNBlock(in_channels, out_channels, kernel_size, stride, padding)
            layers.append(block)
        in_channels = out_channels
        return nn.Sequential(*layers)
    
    def _make_resnet_layers(self, res_block_configs, quantTrue=False):
        layers = []
        for block_config in res_block_configs:
            in_channels = block_config['in_channels']
            out_channels = block_config['out_channels']
            num_blocks = block_config['num_blocks']
            stride = block_config['stride']
            
            layers.append(self._make_layer(ResBlock, in_channels, out_channels, num_blocks, stride, quantTrue=quantTrue))
            
            # Update in_channels for the next set of blocks
            in_channels = out_channels * ResBlock.expansion
        return nn.Sequential(*layers), in_channels

    def _make_layer(self, block, in_channels, out_channels, num_blocks, stride, quantTrue = False):
        strides = [stride] + [1]*(num_blocks-1)  # First block might have a stride to downsample
        blocks = []
        for stride in strides:
            blocks.append(block(in_channels, out_channels, stride, quantTrue=quantTrue))
            in_channels = out_channels * block.expansion  # Update in_channels for the next block
        return nn.Sequential(*blocks)

    def _make_fc_layers(self, prev_features, fc_layer_configs, num_classes):
        layers = nn.ModuleList()
        if fc_layer_configs is not None:
            for fc_layer in fc_layer_configs:
                layers.append(nn.Linear(prev_features, fc_layer['out_features']))
                prev_features = fc_layer['out_features']
            layers.append(nn.Linear(prev_features, num_classes))
        return layers

    def forward(self, x):
        x = self.features(x)
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        for layer in self.classifier:
            if isinstance(layer, nn.Linear):
                x = F.relu(layer(x))
            else:
                x = layer(x)
        return x
    

class QuantizablePrunableCNN(CNN):
    def __init__(self, *args, **kwargs):
        super(QuantizablePrunableCNN, self).__init__(*args, **kwargs)
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()
    
    def forward(self, x):
        x = self.quant(x)
        x = super().forward(x)
        x = self.dequant(x)
        return x
    
    def prune_weights(self, current_sparsity, target_sparsity=0.5):
        for module in self.modules():
            if isinstance(module, (CNNBlock, ResBlock)):
                module.prune_weights(current_sparsity, target_sparsity)

    def convert_to_sparse(self):
        for module in self.modules():
            if isinstance(module, (CNNBlock, ResBlock)):
                module.to_sparse()



# ===== Define and set HyperParameters =====


## DataLoader
TRAIN_BATCH_SIZE = 64  # training batch size
VAL_BATCH_SIZE = 50  # validation batch size
NUM_WORKERS = 8  # number of workers for DataLoader

## Model
### CNN
conv_layer_configs = [
    {'in_channels': 3, 'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 1},
    {'in_channels': 32, 'out_channels': 64, 'kernel_size': 3, 'stride': 1, 'padding': 1},
    {'in_channels': 64, 'out_channels': 128, 'kernel_size': 3, 'stride': 2, 'padding': 1}
]

fc_layer_configs = [
    {'out_features': 256},
    {'out_features': 128}
]

### ResNet
res_block_configs = [
    {'in_channels': 3, 'out_channels': 16, 'stride': 1, 'num_blocks': 3},
    {'in_channels': 16, 'out_channels': 32, 'stride': 2, 'num_blocks': 3},
    {'in_channels': 32, 'out_channels': 64, 'stride': 2, 'num_blocks': 3}
]

## Optimizer and scheduler
INITIAL_LR = 0.1  # initial learning rate
MOMENTUM = 0.9  # momentum for optimizer
REG = 1e-4  # L2 regularization strength
LR_PATIENCE = 5  # Patience for ReduceLROnPlateau scheduler
LR_FACTOR = 0.25  # Factor by which the learning rate will be reduced

## Training
EPOCHS = 5  # total number of training epochs
CHECKPOINT_FOLDER = "./saved_models"  # folder where models are saved

## Pruning and Quantization
ENABLE_QUANTIZATION = False
ENABLE_PRUNING = True
pruning_epochs = 3
starting_sparsity = 0.1
target_sparsity = 0.5

# ===== Set up preprocessing functions =====


## specify preprocessing function
mean = [0.4914, 0.4822, 0.4465]
std = [0.2023, 0.1994, 0.2010]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])


# ===== Set up dataset and dataloader =====


DATA_ROOT = "./data"

## construct dataset
train_set = CIFAR10(
    root = DATA_ROOT, 
    train = True,
    download = True,
    transform = transform_train
)
val_set = CIFAR10(
    root = DATA_ROOT, 
    train = False, 
    download = True,
    transform = transform_val
)

## construct dataloader
train_loader = DataLoader(
    train_set, 
    batch_size= TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)
val_loader = DataLoader(
    val_set, 
    batch_size=VAL_BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)


# ===== Instantiate your SimpleNN model and deploy it to device =====


## specify the device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = QuantizablePrunableCNN(architecture_type='resnet', quantTrue=ENABLE_QUANTIZATION, res_block_configs=res_block_configs, num_classes=10) 
# net = QuantizableCNN(architecture_type='cnn', quantTrue=ENABLE_QUANTIZATION, conv_layer_configs=conv_layer_configs, fc_layer_configs=fc_layer_configs, num_classes=10)


if ENABLE_QUANTIZATION:
    my_qconfig = QConfig(
    activation=torch.quantization.default_observer.with_args(dtype=torch.quint8),
    weight=torch.quantization.default_weight_observer.with_args(dtype=torch.qint8)
    )
    net.qconfig = my_qconfig
    torch.quantization.prepare_qat(net, inplace=True)

# deploy the network to device
net.to(device)

print(next(net.parameters()).device)

Files already downloaded and verified
Files already downloaded and verified
cuda:0


In [2]:
def make_pruning_schedule(pruning_epochs, total_epochs, start_sparsity, final_sparsity):
    sparsity_levels = np.linspace(start_sparsity, final_sparsity, num=pruning_epochs)
    for i in range(total_epochs - pruning_epochs):
        sparsity_levels = np.append(sparsity_levels, final_sparsity)
    return sparsity_levels

# ===== Set up the loss function and optimizer =====


## loss function
criterion = nn.CrossEntropyLoss() 

## Add optimizer
optimizer = optim.SGD(net.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=LR_PATIENCE, factor=LR_FACTOR)

pruning_schedule = make_pruning_schedule(pruning_epochs, EPOCHS, starting_sparsity, target_sparsity)
print(pruning_schedule)

# ===== Start the training process =====


best_val_acc = 0

print("==> Training starts!")
print("="*50)
for i in range(0, EPOCHS):    
    ## switch to train mode
    net.train()

    ## print the Epoch and learning rate
    current_learning_rate = optimizer.param_groups[0]['lr']
    print(f"Epoch {i}: with learning rate {current_learning_rate}")
    
    total_examples = 0
    correct_examples = 0
    train_loss = 0
    
    ## Train the model
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        ### copy inputs to device
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        ### compute the output and loss
        outputs = net(inputs)
        loss = criterion(outputs, targets.long())
        
        ### zero the gradient
        optimizer.zero_grad()
        
        ### backpropagation
        loss.backward()
        
        ### apply gradient and update the weights
        optimizer.step()
        
        ### count the number of correctly predicted samples in the current batch
        _, predicted = torch.max(outputs.data, 1)
        total_examples += targets.size(0)
        correct_examples += (predicted == targets).sum().item()
        train_loss += loss.item()*inputs.size(0)
                
    avg_loss = train_loss / len(train_loader)
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

    if ENABLE_PRUNING:
        current_sparsity = pruning_schedule[i-1] if i > 0 else 0
        target_sparsity = pruning_schedule[i]
        net.prune_weights(current_sparsity, target_sparsity)
        print(f"Pruning to {target_sparsity:.2%} sparsity level")

    ## Validate on the validation dataset
    ## switch to eval mode
    net.eval()

    total_examples = 0
    correct_examples = 0
    val_loss = 0

    ## disable gradient during validation, which can save GPU memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            ### copy inputs to device
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            ### compute the output and loss
            outputs = net(inputs)
            loss = criterion(outputs, targets.long())
            
            ### count the number of correctly predicted samples in the current batch
            _, predicted = torch.max(outputs.data, 1)
            total_examples += targets.size(0)
            correct_examples += (predicted == targets).sum().item()
            val_loss += loss.item() * inputs.size(0)

    avg_loss = val_loss / len(val_loader)
    avg_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))

    ## decay learning rate
    previous_learning_rate = current_learning_rate
    scheduler.step(val_loss)
    current_learning_rate = optimizer.param_groups[0]['lr']
    if previous_learning_rate != current_learning_rate:
        print(f"Learning rate decayed to {current_learning_rate}")
    
    ## save the model checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        if not os.path.exists(CHECKPOINT_FOLDER):
           os.makedirs(CHECKPOINT_FOLDER)
        print("Saving ...")
        state = {'state_dict': net.state_dict(),
                'epoch': i,
                'lr': current_learning_rate}
        saveName = 'CNN_quantizeTrained' if ENABLE_QUANTIZATION else 'CNN'
        torch.save(state, os.path.join(CHECKPOINT_FOLDER, saveName + '.pth'))
    print('')

print("="*50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

[0.1 0.3 0.5 0.5 0.5]
==> Training starts!
Epoch 0: with learning rate 0.1
Training loss: 111.9887, Training accuracy: 0.3638
Pruning Ratio:0.9
Pruning Ratio:0.9
Pruning Ratio:0.9
Pruning Ratio:0.9
Pruning Ratio:0.9
Pruning Ratio:0.9
Pruning Ratio:0.9
Pruning Ratio:0.9
Pruning Ratio:0.9
Pruning to 10.00% sparsity level
Validation loss: 150.9558, Validation accuracy: 0.1055
Saving ...

Epoch 1: with learning rate 0.1
Training loss: 84.8208, Training accuracy: 0.5188
Pruning Ratio:0.6666666666666666
Pruning Ratio:0.6666666666666666
Pruning Ratio:0.6666666666666666
Pruning Ratio:0.6666666666666666
Pruning Ratio:0.6666666666666666
Pruning Ratio:0.6666666666666666
Pruning Ratio:0.6666666666666666
Pruning Ratio:0.6666666666666666
Pruning Ratio:0.6666666666666666
Pruning to 30.00% sparsity level
Validation loss: 229.4842, Validation accuracy: 0.1198
Saving ...

Epoch 2: with learning rate 0.1
Training loss: 77.0758, Training accuracy: 0.5676
Pruning Ratio:0.2857142857142857
Pruning Ratio:0.28

In [3]:
def check_sparsity(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            weight = module.weight.data
            if weight.is_sparse:
                weight = weight.to_dense()
            weight = weight.cpu()
            sparsity = float(torch.sum(weight == 0)) / float(weight.nelement())
            print(f"Sparsity in {name}: {sparsity * 100:.2f}%")

check_sparsity(net)

if ENABLE_PRUNING:
    net.convert_to_sparse()
    print("Converting to sparse model")

Sparsity in init_conv: 0.00%
Sparsity in features.0.0.conv1: 97.69%
Sparsity in features.0.0.conv2: 97.61%
Sparsity in features.0.0.shortcut.0: 0.00%
Sparsity in features.0.1.conv1: 97.61%
Sparsity in features.0.1.conv2: 97.61%
Sparsity in features.0.2.conv1: 97.61%
Sparsity in features.0.2.conv2: 97.61%
Sparsity in features.1.0.conv1: 97.61%
Sparsity in features.1.0.conv2: 97.62%
Sparsity in features.1.0.shortcut.0: 0.00%
Sparsity in features.1.1.conv1: 97.62%
Sparsity in features.1.1.conv2: 97.62%
Sparsity in features.1.2.conv1: 97.62%
Sparsity in features.1.2.conv2: 97.62%
Sparsity in features.2.0.conv1: 97.62%
Sparsity in features.2.0.conv2: 97.62%
Sparsity in features.2.0.shortcut.0: 0.00%
Sparsity in features.2.1.conv1: 97.62%
Sparsity in features.2.1.conv2: 97.62%
Sparsity in features.2.2.conv1: 97.62%
Sparsity in features.2.2.conv2: 97.62%
Converting to sparse model


In [4]:
quantizedModel = True
net.to('cpu')
device = 'cpu'
torch.quantization.convert(net.eval(), inplace=True)

saveName = 'CNN' + '_quantized' if quantizedModel else 'CNN'
torch.save(net.state_dict(), os.path.join(CHECKPOINT_FOLDER, saveName + '.pth'))

# Check the size of the quantized model
model_size_bytes = os.path.getsize(CHECKPOINT_FOLDER + '/' + saveName + '.pth')
model_size_mb = model_size_bytes / (1024 * 1024)  # Convert bytes to megabytes
print(f"Quantized Model Size: {model_size_mb:.2f} MB")

Quantized Model Size: 2.32 MB


In [5]:
def evaluate_model(model, data_loader):
    model.eval()
    total = 0
    correct = 0
    loss = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss += criterion(outputs, labels).item() * images.size(0)
    
    avg_loss = loss / total
    accuracy = 100 * correct / total
    print(f'Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')
    return accuracy

evaluate_model(net, val_loader)

TypeError: cannot assign 'torch.FloatTensor' as parameter 'weight' (torch.nn.Parameter or None expected)